You are given a list of integers, A, and another integer s. Write an algorithm that outputs all the pairs in A that equal s.
For example, if
A = [7, -2, 8, 2, 6, 4, -7, 2, 1, 3, -3] and s = 4
the algorithm should output: (7, -3), (-2, 6), (2, 2), (3, 1).

In [102]:
def findPairs(A,s):
    pairs=[]
    A.sort()
    start=0
    end=len(A)-1
    
    while start<end:
     
        if A[start]+A[end]==s: 
            pairs.append((A[start],A[end]))
            start+=1
    
        elif A[start]+A[end]<s:
            start+=1
            
        else:
            end-=1
            
    return pairs

In [103]:
A = [7, -2, 8, 2, 6, 4, -7, 2, 1, 3, -3]
pairs=findPairs(A,9)

In [104]:
pairs

[(1, 8), (2, 7), (2, 7), (3, 6)]